In [1]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate


llm = OpenAI(model_name="gpt-3.5-turbo-1106")
# llm.predict("개발할 수 있읅까 내가?")

/Users/jangtaehun/Desktop/K_Food/env/lib/python3.10/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/jangtaehun/Desktop/K_Food/env/lib/python3.10/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [ ]:
chat = ChatOpenAI(temperature=0.1)

template = PromptTemplate.from_template('{food}에 대해 알려줘',)
prompt = template.format(food='불고기')
chat.predict(prompt)



/Users/jangtaehun/Desktop/K_Food/env/lib/python3.10/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/jangtaehun/Desktop/K_Food/env/lib/python3.10/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


'불고기에 대해 알려줘'

In [18]:
template = ChatPromptTemplate.from_messages([
    ("system", "너는 외국인에게 한식을 알려주는 전문가야. 항상 영어로만 답해줘야해. 그리고 음식에 관한 정보로 항상 매우 정확해야해. 한식 이름을 알려주면 한식을 어떻게 발음하는지, 어떤 재료가 들어가는지, 어떤 맛인지 어떻게 먹는지 자세하게 알려줘야해. 그리고 추가적으로 알러지를 일으킬 수 있는 지에 대해서도 알려줘."),
    # ('ai', 'message')
    ('human', '{food}'),
])

prompt = template.format_messages(
    food='불고기'
)

chat.predict_messages(prompt)

AIMessage(content='Bulgogi is a popular Korean dish made with thinly sliced beef marinated in a mixture of soy sauce, sugar, sesame oil, garlic, and other ingredients. The name "Bulgogi" is pronounced as "bool-goh-gee." \n\nThe marinated beef is usually grilled or pan-fried, giving it a delicious sweet and savory flavor. Bulgogi is often served with rice, lettuce leaves, and various side dishes such as kimchi. To enjoy Bulgogi, you can wrap a piece of the grilled beef in a lettuce leaf, add some rice and ssamjang (spicy paste), and then eat it in one bite.\n\nIt\'s important to note that Bulgogi may contain soy sauce, which can be a common allergen for some individuals. If you have a soy allergy, it\'s best to check with the restaurant or chef to ensure that the dish is safe for you to consume.')

In [29]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split('.')
        items = [item.replace("'", "") for item in items]
        return list(map(str.strip, items))
    

p = CommaOutputParser()
p.parse('얀, 녕, 나, 김, 쫀, 떡')

['얀, 녕, 나, 김, 쫀, 떡']

In [30]:
template = ChatPromptTemplate.from_messages([
    ('system', '너는 외국인에게 한식을 알려주는 전문가야. 항상 영어로만 답해줘야해. 그리고 음식에 관한 정보로 항상 매우 정확해야해. 한식 이름을 알려주면 한식을 어떻게 발음하는지, 어떤 재료가 들어가는지, 어떤 맛인지 어떻게 먹는지 자세하게 알려줘야해. 그리고 추가적으로 알러지를 일으킬 수 있는 지에 대해서도 알려줘. list ype이 아닌 것으로는 답하지 말고  comma로 구분되는 list type으로 말해줘'),
    ('human', '{food}'),
]
)

prompt = template.format_messages(
    food='불고기'
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()
p.parse(result.content)

['Bulgogi is pronounced as "bool-goh-gee',
 '" It is a popular Korean dish made with thinly sliced beef marinated in a mixture of soy sauce, sugar, sesame oil, garlic, and pepper',
 'Bulgogi is known for its sweet and savory flavor profile',
 'It is typically grilled or stir-fried and served with rice, lettuce leaves, and various side dishes',
 'Please note that bulgogi may contain soy and sesame, which are common allergens for some individuals',
 '']

In [3]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split('.')
        items = [item.replace("'", "") for item in items]
        return list(map(str.strip, items))

In [ ]:
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

chef_template = ChatPromptTemplate.from_messages([
    ('system', '너는 외국인에게 한식을 알려주는 전문가야. 항상 영어로만 답해줘야해. 그리고 음식에 관한 정보로 항상 매우 정확해야해. 한식 이름을 알려주면 한식을 어떻게 발음하는지, 어떤 재료가 들어가는지, 어떤 맛인지 어떻게 먹는지 자세하게 알려줘야해. 그리고 추가적으로 알러지를 일으킬 수 있는 지에 대해서도 알려줘. list ype이 아닌 것으로는 답하지 말고  comma로 구분되는 list type으로 말해줘.'),
    ('human', '{cuisine}'),
])

chef_chain = chef_template | chat | CommaOutputParser()
# chain_2 = template_2 | chat | CommaOutputParser()
# all = chain | chain_2 | outputparser

chef_chain.invoke({
    "cuisine": '감자탕'
})

Gam-ja-tang is a hearty Korean stew made with pork spine, potatoes, vegetables, and seasonings. It has a rich and savory flavor with a slightly spicy kick. To enjoy gamjatang, simply scoop out the tender meat and vegetables from the broth and eat them with a bowl of rice. Be careful, as gamjatang may contain allergens such as pork and spicy seasonings.

['Gam-ja-tang is a hearty Korean stew made with pork spine, potatoes, vegetables, and seasonings',
 'It has a rich and savory flavor with a slightly spicy kick',
 'To enjoy gamjatang, simply scoop out the tender meat and vegetables from the broth and eat them with a bowl of rice',
 'Be careful, as gamjatang may contain allergens such as pork and spicy seasonings',
 '']

In [9]:
examples = [
    {
        "question": "불고기",
        "answer": """
        Pronounced: "bool-goh-gee"
        Explain:
        Bulgogi is a popular Korean dish made with thinly sliced beef marinated in a mixture of soy sauce, sugar, sesame oil, garlic, and pepper.
        It is known for its sweet and savory flavor profile. Bulgogi is typically grilled or pan-fried and often served with rice, lettuce leaves, and ssamjang (a spicy dipping sauce).

        Allergy:
        Please note that bulgogi may contain soy and sesame, which are common allergens for some individuals.
        """
    },
    {
        "question": "순대국",
        "answer": """
        Pronounced: "soon-dae-guk"
        Explain:
        Soondae guk is a traditional Korean soup made with blood sausage (soondae), vegetables, and sometimes noodles.
        The soup has a rich and savory flavor, with the blood sausage providing a unique texture.
        It is often enjoyed as a comforting and hearty meal, especially during colder months.

        Allergy:
        Please note that soondae guk contains blood sausage, which may be an allergen for some individuals.
        """
    },
    {
        "question": "감자탕",
        "answer": """
        Pronounced: "gam-ja-tang"
        Explain:
        Gamjatang is a traditional Korean soup made with pork spine, potatoes, and a variety of vegetables.
        The soup has a rich and hearty flavor, with a mildly spicy broth seasoned with red chili paste, garlic, and fermented soybean paste.
        It is often enjoyed as a comforting meal and is especially popular during colder seasons or as a late-night dish.

        Allergy:
        Please note that gamjatang contains pork and soybean-based seasonings, which may be allergens for some individuals.
        """
    }

]


In [ ]:
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import load_prompt

# prompt = load_prompt("./prompt.json")
chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])



example_template = """
    Human: {question}
    AI: {answer}
"""

example_prompt = ChatPromptTemplate.from_messages([
    ("human: {question}"),
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 외국인에게 음식에 대해 알려주는 음식 설명 전문가야. 짧게 두 문장으로 설명해줘. 여러 개의 음식이 들어오면 둘 다 알려줘"),
    example_prompt,
    ("human", "{food}")
])



chain = final_prompt | chat

chain.invoke(
    {
        "food": "제육볶음"
    }
)

Pronounced: "je-yook-bokkeum"
Explain:
Jeyuk bokkeum is a popular Korean stir-fried pork dish marinated in a spicy-sweet sauce made with gochujang (red chili paste), soy sauce, garlic, and sugar.
It is often cooked with vegetables like onions and green peppers and served with rice, making it a flavorful and satisfying meal.

Allergy:
Please note that jeyuk bokkeum contains pork and soy-based ingredients, which may be allergens for some individuals.

AIMessageChunk(content='Pronounced: "je-yook-bokkeum"\nExplain:\nJeyuk bokkeum is a popular Korean stir-fried pork dish marinated in a spicy-sweet sauce made with gochujang (red chili paste), soy sauce, garlic, and sugar.\nIt is often cooked with vegetables like onions and green peppers and served with rice, making it a flavorful and satisfying meal.\n\nAllergy:\nPlease note that jeyuk bokkeum contains pork and soy-based ingredients, which may be allergens for some individuals.')